In [1]:
!gdown --id 1kIX6ijpz56dBoqHrl_M0X6MGRgF5_cu3

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1kIX6ijpz56dBoqHrl_M0X6MGRgF5_cu3
To: /content/thaisum.csv
100% 2.94G/2.94G [00:12<00:00, 235MB/s]


In [2]:
!gdown --id 1J2XVx41wDvlcCeXHqmsSp1lSvwXtiy5i

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1J2XVx41wDvlcCeXHqmsSp1lSvwXtiy5i
To: /content/bert_only_weight.pt
100% 711M/711M [00:03<00:00, 207MB/s]


In [4]:
!pip install pytorch_transformers

     |████████████████████████████████| 176 kB 7.4 MB/s 
     |████████████████████████████████| 1.2 MB 70.9 MB/s 
     |████████████████████████████████| 132 kB 70.5 MB/s 
     |████████████████████████████████| 880 kB 62.1 MB/s 
     |████████████████████████████████| 8.8 MB 51.6 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 138 kB 78.6 MB/s 
     |████████████████████████████████| 127 kB 76.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=93299f6aaba81091fd7d6c44dff81b2c08cad768e108f3b8e25081d915222b54
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packa

In [5]:
import torch
import torch.nn as nn
from pytorch_transformers import BertModel, BertConfig

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time

# specify GPU
device = torch.device("cuda")

In [7]:
class Bert(nn.Module):
    def __init__(self, temp_dir='temp', finetune=True):
        super(Bert, self).__init__()
        self.model = BertModel.from_pretrained('bert-base-multilingual-cased', cache_dir=temp_dir)

        self.finetune = finetune

    def forward(self, x, segs, mask):
        if(self.finetune):
            top_vec, _ = self.model(x, segs, attention_mask=None)
        else:
            self.eval()
            with torch.no_grad():
                top_vec, _ = self.model(x, segs, attention_mask=None)
        return top_vec

In [8]:
bert = Bert()
bert.load_state_dict(torch.load('/content/bert_only_weight.pt', map_location=lambda storage, loc: storage))

<All keys matched successfully>

In [9]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, n_classes):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,n_classes)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x